### Importing Libraries for Data Analysis and Visualization

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Loading Seasonal Fare Data by Year
 This code reads fare product data for different seasons and years from individual text files into a dictionary of DataFrames (`fares_data`). Each key represents a specific season and year (e.g., `fares_Spring2023`), and the corresponding value is a DataFrame containing fare data for that period.


In [3]:
years = [2023, 2024]
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
fares_data = {}

for year in years:
    for season in seasons:
        
        file_path = f'/Users/bhuvan/Documents/VS code/DS701 project/Datasets/{season}{year}/fare_products.txt'
        if file_path != '/Users/bhuvan/Documents/VS code/DS701 project/Datasets/Winter2024/fare_products.txt':
            fares_data[f'fares_{season}{year}'] = pd.read_csv(file_path, delimiter=',')
            fares_data[f'fares_{season}{year}']['season'] = f'{season}'
            fares_data[f'fares_{season}{year}']['year'] = f'{year}'
        else:
            continue

### Loading Seasonal Fare Rules Data by Year

This code loads fare rules data from text files for different seasons and years into a dictionary of DataFrames (`fares_rules_data`). Each key in the dictionary represents a unique season and year (e.g., `fares_rules_Spring2023`), allowing for organized and accessible seasonal data analysis.

In [4]:
years = [2023, 2024]
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
fares_rules_data = {}

for year in years:
    for season in seasons:
        
        file_path = f'/Users/bhuvan/Documents/VS code/DS701 project/Datasets/{season}{year}/fare_leg_rules.txt'
        if file_path != '/Users/bhuvan/Documents/VS code/DS701 project/Datasets/Winter2024/fare_leg_rules.txt':
            fares_rules_data[f'fares_rules_{season}{year}'] = pd.read_csv(file_path, delimiter=',')
        else:
            continue

In [ ]:
routes_data = pd.read_csv('/Users/bhuvan/Documents/DS701 project/Datasets/Spring2023/routes.txt', delimiter=',')
routes_data = routes_data[routes_data['route_desc'] == 'Commuter Rail']
routes_data = routes_data.dropna(axis=1, how='any')
routes_data = routes_data.drop('route_text_color', axis=1, inplace=False)
#routes_data.info()

### Combining Seasonal Fare DataFrames

This code combines all the seasonal fare DataFrames from the `fares_data` dictionary into a single DataFrame (`combined_fares_df`) and provides an overview of its structure.

In [ ]:
combined_fares_df = pd.concat(fares_data.values(), ignore_index=True)
combined_fares_df.info()

### Combining Seasonal Fare Rules DataFrames

This code merges all the seasonal fare rules DataFrames from the `fares_rules_data` dictionary into a single DataFrame (`combined_fares_rules_df`) and provides a summary of its structure.

In [ ]:
combined_fares_rules_df = pd.concat(fares_rules_data.values(), ignore_index=True)
combined_fares_rules_df.info()

### Dropping Unnecessary Columns from Combined Fare Rules DataFrame

This code removes specific columns from the `combined_fares_rules_df` DataFrame and displays a summary of the updated DataFrame structure.

In [ ]:
combined_fares_rules_df = combined_fares_rules_df.drop(columns=['transfer_only', 'from_timeframe_group_id', 'to_timeframe_group_id'])
combined_fares_rules_df.info()

### Merging Combined Fare and Fare Rules DataFrames

This code merges the `combined_fares_df` and `combined_fares_rules_df` DataFrames into a single DataFrame (`combined_fare_result`) based on the `fare_product_id` column and provides a brief summary of this.

In [ ]:
combined_fare_result = pd.merge(combined_fares_df, combined_fares_rules_df, on='fare_product_id', how='inner')
combined_fare_result = pd.merge(combined_fare_result, routes_data, on='network_id', how='left')
combined_fare_result = combined_fare_result.drop(columns=['agency_id', 'route_type', 'route_sort_order', 'route_color'])
combined_fare_result.head()

### Filtering Combined Fare Data for Commuter Rail

This code filters the `combined_fare_result` DataFrame to include only rows related to the 'commuter_rail' and 'cape_flyer' networks.

In [ ]:
combined_fare_result_CR = combined_fare_result[combined_fare_result['network_id'].isin(['commuter_rail', 'cape_flyer'])]
combined_fare_result_CR.info()

In [ ]:
combined_fare_result_CR_cash = combined_fare_result_CR[combined_fare_result_CR['fare_media_id'] == 'cash']
combined_fare_result_CR_cash_1a = combined_fare_result_CR[combined_fare_result_CR['from_area_id'] == 'area_commuter_rail_zone_1a']
combined_fare_result_CR_cash_1a.info()

In [ ]:
combined_fare_result_CR_cash_1a_drop = combined_fare_result_CR_cash_1a.drop_duplicates()
combined_fare_result_CR_cash_1a_drop = combined_fare_result_CR_cash_1a_drop[combined_fare_result_CR_cash_1a_drop['fare_media_id'] == 'cash']
combined_fare_result_CR_cash_1a_drop.info()

In [ ]:
combined_fare_result_CR_cash_1a_drop_group = combined_fare_result_CR_cash_1a_drop.groupby(['year', 'season', 'route_id', 'to_area_id']).size().reset_index(name='count')
combined_fare_result_CR_cash_1a_drop_group.info()

In [26]:
# Define a dictionary specifying the `to_area_id` you want to keep for each `line_id`
to_area_id_mapping = {
    'line-CapeFlyer': 'area_cf_zone_hyannis',  # example
    'line-Fairmount': 'area_commuter_rail_zone_2',  # example
    'line-Fitchburg': 'area_commuter_rail_zone_8',
    'line-Worcester': 'area_commuter_rail_zone_8',
    'line-Franklin': 'area_commuter_rail_zone_6',
    'line-Greenbush': 'area_commuter_rail_zone_6',
    'line-Haverhill': 'area_commuter_rail_zone_7',
    'line-Kingston': 'area_commuter_rail_zone_8',
    'line-Lowell': 'area_commuter_rail_zone_6',
    'line-Middleborough': 'area_commuter_rail_zone_8',
    'line-Needham': 'area_commuter_rail_zone_2',
    'line-Newburyport': 'area_commuter_rail_zone_8',
    'line-Providence': 'area_commuter_rail_zone_10',
}
#combined_fare_result_CR_cash_1a_drop['to_area_id'] = combined_fare_result_CR_cash_1a_drop['line_id'].map(to_area_id_mapping).fillna(combined_fare_result_CR_cash_1a_drop['to_area_id'])
filtered_df = combined_fare_result_CR_cash_1a_drop[
    combined_fare_result_CR_cash_1a_drop.apply(
        lambda row: row['to_area_id'] == to_area_id_mapping.get(row['line_id'], row['to_area_id']),
        axis=1
    )
]
# Perform a left merge to bring in the 'amount' column from combined_fare_result_CR_cash_1a_drop
# Ensure only 'amount' column is brought over to avoid any duplication of columns
merged_df = filtered_df.merge(
    combined_fare_result_CR_cash_1a_drop[['to_area_id', 'amount']],
    on='to_area_id',
    suffixes=('', '_new'),
    how='left'
)

# Update the 'amount' column in filtered_df to match the values from combined_fare_result_CR_cash_1a_drop
merged_df['amount'] = merged_df['amount_new'].combine_first(merged_df['amount'])

# Drop the 'amount_new' column as it was only needed for updating
merged_df = merged_df.drop(columns=['amount_new'])
merged_df = merged_df.drop_duplicates()

# Now merged_df has the updated 'amount' values in the 'amount' column.

In [ ]:
merged_df_ppt = merged_df[['route_id', 'amount']]
merged_df_ppt = merged_df_ppt.drop_duplicates()
merged_df_ppt = merged_df_ppt.reset_index(drop=True)
merged_df_ppt = merged_df_ppt.sort_values(by='amount', ascending=False)
merged_df_ppt = merged_df_ppt.reset_index(drop=True)
merged_df_ppt['amount'] = merged_df_ppt['amount'].apply(lambda x: f"${x:.2f}")
merged_df_ppt.head(15)

### Analyzing Average Fare Cost Changes Over Time for Commuter Rail

This code groups the filtered fare data by season, year, and fare product name, calculates the average fare amount, and visualizes fare changes over time.

In [ ]:
combined_fare_result_CR_group = combined_fare_result_CR_cash.groupby(['season', 'year', 'fare_product_name'])['amount'].mean().reset_index()
sns.lineplot(data=combined_fare_result_CR_group, x='season', y='amount', hue='year')
plt.title('Average Fare Cost Changes Over Time')
plt.show()

In [ ]:
# Assuming your dataframe is named 'df'
merged_df['season_year'] = merged_df['season'] + ' ' + merged_df['year'].astype(str)

# Sort the data by 'year' and 'season' to ensure the timeline is correctly ordered
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
merged_df['season'] = pd.Categorical(merged_df['season'], categories=season_order, ordered=True)
df = merged_df.sort_values(['year', 'season'])

# Get the unique route_ids
route_ids = merged_df['route_id'].unique()

# Set up a color map for distinguishing different route IDs
colors = plt.cm.tab20.colors

# Loop through each route_id and plot
for i, route_id in enumerate(route_ids):
    # Filter the data for the current route_id
    route_data = merged_df[merged_df['route_id'] == route_id]
    
    # Create a new figure for each route
    plt.figure(figsize=(10, 6))
    
    # Scatter plot with x as season-year and y as amount
    plt.scatter(route_data['season_year'], route_data['amount'], color=colors[i % len(colors)], label=route_id)
    
    # Line plot to connect the points
    plt.plot(route_data['season_year'], route_data['amount'], color=colors[i % len(colors)], linestyle='-', marker='o')
    
    # Add labels and title
    plt.xlabel("Season & Year")
    plt.ylabel("Amount")
    plt.title(f"Fare Amount Over Time for Route ID: {route_id}")
    
    # Rotate x-axis labels for readability
    plt.xticks(rotation=45)
    
    # Add legend and grid
    plt.legend()
    plt.grid(True)
    
    # Show plot
    plt.tight_layout()
    plt.show()

## -----------------------------------------------------------------------------------